A web crawler is set up to pull sale data off of p24. 
# SET THE CSV FILE NAME IN LAST CELL

In [3]:
import requests
from bs4 import BeautifulSoup
import csv
from tqdm import tqdm 

# Define the number of pages
num_iterations = 6

pricelist = []
sizelist = []
locationlist = []

#tqdm(range(num_iterations), desc="Scraping Progress")

# Create a tqdm progress bar
for x in range(5):
    p24 = f"https://www.property24.com/apartments-for-sale/gauteng/1/p{x}"
    req = requests.get(p24)
    soup = BeautifulSoup(req.text, "html.parser")

    for price in soup.find_all("span", "p24_price"):
        pricelist.append(price.text)   
    
    for size in soup.find_all("span", "p24_size"):
        sizelist.append(size.text)

    for location in soup.find_all("span", "p24_location"):
        locationlist.append(location.text)

pricelist = [price.strip().replace("\n", "").replace("\r", "").replace(" ", "").replace("R", "").replace("POA", "0") for price in pricelist]  

sizelist = [size.strip().replace("\n\n", "").replace("m²\n", "").replace("m² ", "").replace(" ha", "0").replace(" m²", "").replace(" ",  "").replace(".", "") for size in sizelist]  

csvdata = [[int(addr), int(price), str(location)] for addr, price, location in set(zip(pricelist, sizelist, locationlist))]

print(f"CSVDATA: {len(csvdata), csvdata}")


CSVDATA: (76, [[240000, 135, 'Melrose North'], [899000, 104, 'Klippoortje'], [720000, 97, 'Floracliffe'], [950000, 82, 'Hatfield'], [799000, 76, 'Klippoortje'], [650000, 84, 'Crowthorne AH'], [1150000, 84, 'Kyalami Hills'], [1696000, 99, 'Waterfall'], [895000, 104, 'Knysna Central'], [1495000, 81, 'Glen Austin AH'], [1020000, 48, 'Florida Park'], [12995, 140, 'Eastcliff'], [795000, 54, 'Chantelle'], [330000, 120, 'Bryanston'], [499000, 73, 'Bryanston'], [930000, 70, 'Norton Park'], [680000, 44, 'Halfway Gardens'], [700000, 92, 'New Redruth'], [380000, 342, 'Melrose'], [895000, 35, 'Johannesburg Central'], [500000, 114, 'Northgate'], [1375000, 34, 'Sandton Central'], [2189000, 83, 'Douglasdale'], [2450000, 56, 'Pretoria North'], [580000, 123, 'Alveda'], [1375000, 54, 'Kyalami Hills'], [680000, 47, 'Melrose North'], [650000, 36, 'Glen Austin AH'], [665000, 75, 'Sagewood'], [1495000, 84, 'Hatfield'], [1200000, 89, 'Rynfield'], [720000, 101, 'Eastcliff'], [330000, 135, 'Oakdene'], [795000,

Using the propattr class the median and sd are calculated for the data. 

The data should be sorted by area name to avoid over use of the api. 

The data is sorted by name and pulled into its own list to find the co-ordinates using Bings Map Api.

In [ ]:
import geocoder
from tqdm import tqdm

commonlocations = []


for data in csvdata:
    if data[1] not in commonlocations:
        commonlocations.append(data[1])

print(f"SIZE: {len(commonlocations)} {commonlocations}")

common_coords = []

failed = []

for addr in tqdm(commonlocations, desc="Finding Coordinates"):
    try:
        g = geocoder.bing(f'{addr + "northern cape" + "South Africa"}', key="YOURKEY")
        
        # Check if the geocoding request was successful
        if g.ok:
            results = g.json
            common_coords.append([results["lat"], results["lng"], addr])
        else:
            # Handle the case where the geocoding request failed
            print(f"Geocoding request failed for address: {addr}")
            failed.append(addr)
    except Exception as e:
        # Handle any exceptions that may occur during the geocoding request
        print(f"An error occurred while geocoding address: {addr}")
        print(f"Error details: {str(e)}")

Enter data into csv for locations, price and coords.

In [4]:
import os
import csv

# Specify the target directory path
target_directory = r'C:\GitHub\Land-value-GIS\backups'

# Create the target directory if it doesn't exist
#if not os.path.exists(target_directory):
#    os.makedirs(target_directory)

# Define the file paths within the "backups" folder
csvdata_file_path = os.path.join(target_directory, "house_price_and_area_test.csv")
common_coords_file_path = os.path.join(target_directory, "northern_cape_area_coords.csv")

# Write to "prop_gauteng_locations_price.csv"
with open(csvdata_file_path, "w") as r:
    writer = csv.writer(r)
    writer.writerows(csvdata)

# Write to "gauteng_area_coords.csv"
#with open(common_coords_file_path, "w") as w:
#    writer = csv.writer(w)
#    writer.writerows(common_coords)


Move the files into the backups dir